In [1]:
from scipy import integrate
from sympy import *
import numpy as np
import pandas as pd
from collections import Counter
import math

t = symbols('t d')

In [11]:
def FM(A):
    n = len(A)
    lam,vlam = (np.linalg.eig(A))
    k_lam = Counter(lam) # собственные числа и их кратности
    list_b = list(var('b0 b1 b2 b3 b4 b5 b6 b7 b8 b9 b10',real = True))
    var('t',real = True) # полиномы, переменные
    var('lamb')
     
    # правая части уравнений для поиска полиномов
    right = list_b[0]
    for i in range(1, n):
        right += list_b[i]*lamb**i
    right -= exp(t*lamb)
    # генерируем систему
    B = []
    i = 0
    while i < n:
        for x in k_lam:
            if im(x) >= 0:
                for r in range(k_lam[x]):
                    d_right = right
                    for k in range(r):
                        d_right = diff(d_right,lamb)
                        
                    d_right = d_right.subs(lamb,x)
                    if im(x) == 0:
                        B.append(simplify(d_right))
                    else:
                        B.append(simplify(re(d_right)))
                        B.append(simplify(im(d_right)))
            i += k_lam[x]
            
            
    ans, = simplify(linsolve(B,list_b))
    An = []
    for i in range(n):
        Ak = np.eye(n)
        for j in range(i):
            Ak = np.matmul(Ak,A)
        An.append(Ak)
    Y = np.zeros((n,n))
    for i in range(n):
        Y = Y + ans[i]*Matrix(An[i])
    Y = simplify(Y)
    return (Y)

In [23]:
P = np.matrix([
    [4, 2],
    [4, -3],
])
f = Matrix([
    [1],
    [2],
])
R = Matrix([[1*exp(t),-1*exp(t)]])
fi = 0
Y = FM(P)
y = t
T = 1

In [24]:
# проверяем, есть ли на главной диагонали нули
def no0(M, k):
    n =  M.shape[0]
    while k < n and not M[k, k]:
        k += 1
    return k

In [25]:
def Gauss(A,b):
        M = np.hstack((A, b)) # "склейка" матрицы коэффициентов при переменных и вектора свободных членов
        n = M.shape[0] 
        for k in range(n): 
            # убираем нули с главной диагонали, если они есть
            k0 = no0(M, k)
            if k0 != k:
                M[k0], M[k] = M[k].copy(), M[k0]   
            if M[k, k] != 1:
                M[k, :] *= 1 / M[k, k]
            # приямой ход метода Гаусса
            for r in range(k + 1, n):
                M[r, :] -= M[k, :] * M[r, k]
        #обратный ход метода Гаусса
        for k in range(n - 1, 0, -1):
            for r in range(k - 1, -1, -1):
                if M[r, k]:
                    M[r, :] -= M[k, :] * M[r, k]
        if b.shape[1] > 1:
            return np.hsplit(M,2)[1]
        return  (M[:,n])

In [26]:
H = R*Y

g = y - fi - (H*integrate(Y**(-1)*f))[0,0]
n = integrate(H.T*g,t)
n = n.subs(t,T)
D = integrate(H.T*H,t)
D = D.subs(t,T)

x0 = Gauss(D,n)


KeyboardInterrupt: 

In [ ]:
D

In [ ]:
x0

In [ ]:
D.inv()*n